In [12]:
import os, sys

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
my_path = '/content/drive/MyDrive/Colab_Notebooks/additional_package'

#!pip install --target=$my_path package

# 다운 받은 패키지 바로 import할 수 있게 하기
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/additiona_package')

os.chdir("/content/drive/MyDrive/Colab_Notebooks/AI_Base")

##패키지 설치

In [14]:
#!pip install --target=$my_path gdown 
#!pip install --target=$my_path -U torchtext
#!pip install --target=$my_path git+https://github.com/PyTorchLightning/pytorch-lightning
#!pip install  --target=$my_path git+https://github.com/SKT-AI/KoBART#egg=kobart
#!pip install  --target=$my_path transformers==4.8.2 -U

In [ ]:
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart # koBART 
!pip install transformers==4.8.2 -U # transformer
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git # 자동 띄어쓰기

In [17]:
import gdown
import logging
import numpy as np
import pandas as pd
import re
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from dataset import KoBARTSummaryDataset
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
import tensorflow
from pykospacing import Spacing
spacing = Spacing()

In [ ]:
#학습한 모델 

### 모델 로딩

In [18]:
if os.path.exists('kobart_summary/config.json') and os.path.exists('kobart_summary/pytorch_model.bin') :
    print("== Data existed.==")
    pass
else:
    os.system("rm -rf kobary_summary")
    os.system("mkdir kobart_summary")
    url = "https://drive.google.com/uc?id=1H13loH6dS_2c2Z21kaBtgz42QsjkdAwO"
    output = './kobart_summary/config.json'
    print("Download config.json")
    gdown.download(url, output, quiet=False)

    url = "https://drive.google.com/uc?id=1D7BAXK_0faWW39c0ptE3FtROsVRbTNwI"
    output = './kobart_summary/pytorch_model.bin'
    print("Download pytorch_model.bin")
    gdown.download(url, output, quiet=False)

== Data existed.==


In [19]:

from transformers.models.bart import BartForConditionalGeneration


def load_model():
    model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Colab_Notebooks/kobart_summary')
    # tokenizer = get_kobart_tokenizer()
    return model

model = load_model()
tokenizer = get_kobart_tokenizer()

using cached model


In [20]:
civil_law= ["제578조(경매와 매도인의 담보책임) ①경매의 경우에는 경락인은 전8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금감액의 청구를 할 수 있다.②전항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금전부나 일부의 반환을 청구할 수 있다.③전2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다."
, "제579조(채권매매와 매도인의 담보책임) ①채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약당시의 자력을 담보한 것으로 추정한다.②변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 변제기의 자력을 담보한 것으로 추정한다."
,"제688조(수임인의 비용상환청구권 등) ①수임인이 위임사무의 처리에 관하여 필요비를 지출한 때에는 위임인에 대하여 지출한 날 이후의 이자를 청구할 수 있다.②수임인이 위임사무의 처리에 필요한 채무를 부담한 때에는 위임인에게 자기에 갈음하여 이를 변제하게 할 수 있고 그 채무가 변제기에 있지 아니한 때에는 상당한 담보를 제공하게 할 수 있다.  <개정 2014. 12. 30.>③수임인이 위임사무의 처리를 위하여 과실없이 손해를 받은 때에는 위임인에 대하여 그 배상을 청구할 수 있다"
,"제727조(종신정기금계약의 해제) ①정기금채무자가 정기금채무의 원본을 받은 경우에 그 정기금채무의 지급을 해태하거나 기타 의무를 이행하지 아니한 때에는 정기금채권자는 원본의 반환을 청구할 수 있다. 그러나 이미 지급을 받은 채무액에서 그 원본의 이자를 공제한 잔액을 정기금채무자에게 반환하여야 한다.②전항의 규정은 손해배상의 청구에 영향을 미치지 아니한다."
, "제758조(공작물등의 점유자, 소유자의 책임) ①공작물의 설치 또는 보존의 하자로 인하여 타인에게 손해를 가한 때에는 공작물점유자가 손해를 배상할 책임이 있다. 그러나 점유자가 손해의 방지에 필요한 주의를 해태하지 아니한 때에는 그 소유자가 손해를 배상할 책임이 있다.②전항의 규정은 수목의 재식 또는 보존에 하자있는 경우에 준용한다.③제2항의 경우에 점유자 또는 소유자는 그 손해의 원인에 대한 책임있는 자에 대하여 구상권을 행사할 수 있다.",
'제410조(1인의 채권자에 생긴 사항의 효력) ①전조의 규정에 의하여 모든 채권자에게 효력이 있는 사항을 제외하고는 불가분채권자중 1인의 행위나 1인에 관한 사항은 다른 채권자에게 효력이 없다.②불가분채권자 중의 1인과 채무자간에 경개나 면제있는 경우에 채무전부의 이행을 받은 다른 채권자는 그 1인이 권리를 잃지 아니하였으면 그에게 분급할 이익을 채무자에게 상환하여야 한다.'
,'제444조(부탁없는 보증인의 구상권) ①주채무자의 부탁없이 보증인이 된 자가 변제 기타 자기의 출재로 주채무를 소멸하게 한 때에는 주채무자는 그 당시에 이익을 받은 한도에서 배상하여야 한다.②주채무자의 의사에 반하여 보증인이 된 자가 변제 기타 자기의 출재로 주채무를 소멸하게 한 때에는 주채무자는 현존이익의 한도에서 배상하여야 한다.③전항의 경우에 주채무자가 구상한 날 이전에 상계원인이 있음을 주장한 때에는 그 상계로 소멸할 채권은 보증인에게 이전된다.'
,'제594조(환매의 실행) ①매도인은 기간내에 대금과 매매비용을 매수인에게 제공하지 아니하면 환매할 권리를 잃는다.②매수인이나 전득자가 목적물에 대하여 비용을 지출한 때에는 매도인은 제203조의 규정에 의하여 이를 상환하여야 한다. 그러나 유익비에 대하여는 법원은 매도인의 청구에 의하여 상당한 상환기간을 허여할 수 있다.'
,'제484조(대위변제와 채권증서, 담보물) ①채권전부의 대위변제를 받은 채권자는 그 채권에 관한 증서 및 점유한 담보물을 대위자에게 교부하여야 한다.②채권의 일부에 대한 대위변제가 있는 때에는 채권자는 채권증서에 그 대위를 기입하고 자기가 점유한 담보물의 보존에 관하여 대위자의 감독을 받아야 한다.'
,'제13조(피한정후견인의 행위와 동의) ① 가정법원은 피한정후견인이 한정후견인의 동의를 받아야 하는 행위의 범위를 정할 수 있다.② 가정법원은 본인, 배우자, 4촌 이내의 친족, 한정후견인, 한정후견감독인, 검사 또는 지방자치단체의 장의 청구에 의하여 제1항에 따른 한정후견인의 동의를 받아야만 할 수 있는 행위의 범위를 변경할 수 있다.③ 한정후견인의 동의를 필요로 하는 행위에 대하여 한정후견인이 피한정후견인의 이익이 침해될 염려가 있음에도 그 동의를 하지 아니하는 때에는 가정법원은 피한정후견인의 청구에 의하여 한정후견인의 동의를 갈음하는 허가를 할 수 있다.④ 한정후견인의 동의가 필요한 법률행위를 피한정후견인이 한정후견인의 동의 없이 하였을 때에는 그 법률행위를 취소할 수 있다. 다만, 일용품의 구입 등 일상생활에 필요하고 그 대가가 과도하지 아니한 법률행위에 대하여는 그러하지 아니하다.'
,'제829조(부부재산의 약정과 그 변경) ①부부가 혼인성립전에 그 재산에 관하여 따로 약정을 하지 아니한 때에는 그 재산관계는 본관중 다음 각조에 정하는 바에 의한다.②부부가 혼인성립전에 그 재산에 관하여 약정한 때에는 혼인중 이를 변경하지 못한다. 그러나 정당한 사유가 있는 때에는 법원의 허가를 얻어 변경할 수 있다.③전항의 약정에 의하여 부부의 일방이 다른 일방의 재산을 관리하는 경우에 부적당한 관리로 인하여 그 재산을 위태하게 한 때에는 다른 일방은 자기가 관리할 것을 법원에 청구할 수 있고 그 재산이 부부의 공유인 때에는 그 분할을 청구할 수 있다.④부부가 그 재산에 관하여 따로 약정을 한 때에는 혼인성립까지에 그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못한다.⑤제2항, 제3항의 규정이나 약정에 의하여 관리자를 변경하거나 공유재산을 분할하였을 때에는 그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못한다.']

In [21]:
# 한글 영어 숫자를 남겨 놓고 제거

for i in range(len(civil_law)):
  civil_law[i]= re.sub('[^A-Za-z0-9가-힣]', '', civil_law[i])
  civil_law[i]= spacing(civil_law[i])
civil_law

['제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있다 전 항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있다 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다',
 '제579조 채권 매매와 매도인의 담보책임채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약 당시의 자력을 담보한 것으로 추정한 다변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 변제기의 자력을 담보한 것으로 추정한다',
 '제688조 수임인의 비용 상환청구권 등 수임인이 위임사무의 처리에 관하여 필요비를 지출한 때에는 위임인에 대하여 지출한 날 이후의 이자를 청구할 수 있다 수임인이 위임사무의 처리에 필요한 채무를 부담한 때에는 위임인에게 자기에 갈음하여 이를 변제하게 할 수 있고 그 채무가 변제기에 있지 아니한 때에는 상당한 담보를 제공하게 할 수 있다 개정 20141230 수임인이 위임사무의 처리를 위하여 과실 없이 손해를 받은 때에는 위임인에 대하여 그 배상을 청구할 수 있다',
 '제727조종신 정기금 계약의 해제 정기금 채무자가 정기금 채무의원본을 받은 경우에 그 정기금 채무의 지급을 해태하거나 기타의무를 이행하지 아니한 때에는 정기금채권자는 원본의 반환을 청구할 수 있다 그러나 이미 지급을 받은 채 무액에서 그 원본의 이자를 공제한 잔액을 정기금 채무자에게 반환하여야 한다 전 항의 규정은 손해배상의 청구에 영향을 미치지 아니한다',
 '제758조 공작물 등의 점유자 소유자의 책임 공작물의 설치 또는 보존의 하자로 인하여 타인에게 손해를 가한 때에는 공작물 점유자가 손해를 배상할 책임이 있다 그러나 점유자가 손해의 방지에 필요한 주의를 해태하지 아

In [22]:
civil_law=['제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있다. 전 항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있다. 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.',
 '제579조 채권 매매와 매도인의 담보책임채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약 당시의 자력을 담보한 것으로 추정한다. 변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 변제기의 자력을 담보한 것으로 추정한다.',
 '제688조 수임인의 비용 상환청구권 등 수임인이 위임사무의 처리에 관하여 필요비를 지출한 때에는 위임인에 대하여 지출한 날 이후의 이자를 청구할 수 있다. 수임인이 위임사무의 처리에 필요한 채무를 부담한 때에는 위임인에게 자기에 갈음하여 이를 변제하게 할 수 있고, 그 채무가 변제기에 있지 아니한 때에는 상당한 담보를 제공하게 할 수 있다 개정 20141230 수임인이 위임사무의 처리를 위하여 과실 없이 손해를 받은 때에는 위임인에 대하여 그 배상을 청구할 수 있다.',
 '제727조종신 정기금 계약의 해제 정기금 채무자가 정기금 채무의원본을 받은 경우에 그 정기금 채무의 지급을 해태하거나 기타의무를 이행하지 아니한 때에는 정기금채권자는 원본의 반환을 청구할 수 있다. 그러나 이미 지급을 받은 채 무액에서 그 원본의 이자를 공제한 잔액을 정기금 채무자에게 반환하여야 한다. 전 항의 규정은 손해배상의 청구에 영향을 미치지 아니한다.',
 '제758조 공작물 등의 점유자 소유자의 책임 공작물의 설치 또는 보존의 하자로 인하여 타인에게 손해를 가한 때에는 공작물 점유자가 손해를 배상할 책임이 있다. 그러나 점유자가 손해의 방지에 필요한 주의를 해태하지 아니한 때에는 그 소유자가 손해를 배상할 책임이 있다. 전 항의 규정은 수목의 재식 또는 보존에 하자 있는 경우에 준용한다. 제2항의 경우에 점유자 또는 소유자는 그 손해의 원인에 대한 책임 있는 자에 대하여 구상권을 행사할 수 있다.',
 '제410조 1인의 채권자에 생긴 사항의 효력 전조의 규정에 의하여 모든 채권자에게 효력이 있는 사항을 제외하고는 불가분채권자 중 1인의 행위나 1인에 관한 사항은 다른 채권자에게 효력이 없다. 불가분채권자 중의 1인과 채무자 간에 경개나 면제 있는 경우에 채무 전부의 이행을 받은 다른 채권자는 그 1인이 권리를 잃지 아니하였으면 그에게 분급할 이익을 채무자에게 상환하여야 한다.',
 '제444조 부탁 없는 보증인의 구상권 주채무자의 부탁 없이 보증인이 된 자가 변제기타 자기의 출재로 주채무를 소멸하게 한 때에는 주채무자는 그 당시에 이익을 받은 한도에서 배상하여야 한다. 주 채무자의 의사에 반하여 보증인이 된 자가 변제기타 자기의 출재로 주채무를 소멸하게 한 때에는 주채무자는 현존이익의 한도에서 배상하여야 한다. 전 항의 경우에 주채무자가 구상한 날이전에 상계 원인이 있음을 주장한 때에는 그 상계로 소멸할 채권은 보증인에게 이전된다.',
 '제594조 환매의 실행매도인은 기간 내에 대금과 매매비용을 매수인에게 제공하지 아니하면 환매할 권리를 잃는 다매수인이나 전득자가 목적물에 대하여 비용을 지출한 때에는 매도인은 제203조의 규정에 의하여 이를 상환하여야 한다. 그러나 유익비에 대하여는 법원은 매도인의 청구에 의하여 상당한 상환기간을 허여할 수 있다.',
 '제484조 대위변제와 채권증서 담보물 채권 전부의 대위변제를 받은 채권자는 그 채권에 관한 증서 및 점유한 담보물을 대위자에게 교부하여야 한다. 채권의 일부에 대한 대위변제가 있는 때에는 채권자는 채권증서에 그 대위를 기입하고 자기가 점유한 담보물의 보존에 관하여 대위자의 감독을 받아야 한다',
 '제13조 피한 정후견인의 행위와 동의가 정법원은 피한 정후견인이 한정후견인의 동의를 받아야 하는 행위의 범위를 정할 수 있다.가정법원은 본인 배우자 4촌 이내의 친족한 정후견인 한정후견 감독인 검사 또는 지방자치단체의 장의 청구에 의하여 제1항에 따른 한정후견인의 동의를 받아야만 할 수 있는 행위의 범위를 변경할 수 있다. 한정후견인의 동의를 필요로 하는 행위에 대하여 한정후견인이 피한 정후견인의 이익이 침해될 염려가 있음에도 그동의를 하지 아니하는 때에는 가정법 원은 피한 정후견인의 청구에 의하여 한정후견인의 동의를 갈음하는 허가를 할 수 있다. 한정후견인의 동의가 필요한 법률행위를 피한 정후견인이 한정후견인의 동의 없이 하였을 때에는 그 법률행위를 취소할 수 있다. 다만 일용품의 구입 등 일상생활에 필요하고 그 대가가 과도하지 아니한 법률행위에 대하여는 그러하지 아니하다.',
 '제829조 부부 재산의 약정과 그 변경 부부가 혼인성립 전에 그 재산에 관하여 따로 약정을 하지 아니한 때에는 그 재산관계는 본관 중 다음 각 조에 정하는 바에 의한다. 부부가 혼인성립 전에 그 재산에 관하여 약정한 때에는 혼인 중 이를 변경하지 못한다. 그러나 정당한 사유가 있는 때에는 법원의 허가를 얻어 변경할 수 있다. 전 항의약정에 의하여 부부의 일방이 다른 일방의 재산을 관리하는 경우에 부적당한 관리로 인하여 그 재산을 위태하게 한 때에는 다른 일방은 자기가 관리할 것을 법원에 청 구할 수 있고 그 재산이 부부의 공유인 때에는 그 분할을 청구할 수 있다. 부부가 그 재산에 관하여 따로 약정을 한 때에는 혼인 성립까지 에그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못한다. 제2항 제3항의 규정이나 약정에 의하여 관리자를 변경하거나 공유재산을 분할하였을 때에는 그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못한다.']

In [ ]:
#형

In [23]:
outputlist2=[]
for law in civil_law:
  text =  law
  input_ids = tokenizer.encode(text)
  input_ids = torch.tensor(input_ids)
  input_ids = input_ids.unsqueeze(0)
  output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  outputlist2.append(output)
for i in range(len(civil_law)):
  print("원문:", civil_law[i])
  print('\033[91m'+"요약:",outputlist2[i]+'\033[0m')

# CSV 파일로 저장
result_df=pd.DataFrame(data= zip(civil_law,outputlist2), columns= ["원문","요약"])
result_df.to_csv("민법 원문 요약.csv")

원문: 제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있다. 전 항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있다. 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.
요약: 제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있고, 전항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있으며, 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.
원문: 제579조 채권 매매와 매도인의 담보책임채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약 당시의 자력을 담보한 것으로 추정한다. 변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 변제기의 자력을 담보한 것으로 추정한다.
요약: 변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약 당시의 자력을 담보한 것으로 추정한다.
원문: 제688조 수임인의 비용 상환청구권 등 수임인이 위임사무의 처리에 관하여 필요비를 지출한 때에는 위임인에 대하여 지출한 날 이후의 이자를 청구할 수 있다. 수임인이 위임사무의 처리에 필요한 채무를 부담한 때에는 위임인에게 자기에 갈음하여 이를 변제하게 할 수 있고, 그 채무가 변제기에 있지 아니한 때에는 상당한 담보를 제공하게 할 수 있다 개정 20141230 수임인이 위임사무의 처리를 위하여 과실 없이 손해를 받은 때에

#용어 순화 작업

In [24]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import konlpy
from konlpy.tag import Kkma, Komoran, Okt, Hannanum
okt = Okt()
import tensorflow
import keras
#!pip install  git+https://github.com/haven-jeon/PyKoSpacing.git
from pykospacing import Spacing
spacing = Spacing()

### 민법 원문 요약 데이터

In [25]:
civil_law= pd.read_csv("민법 원문 요약.csv")
civil_law.drop(columns = civil_law.columns[0], inplace=True) # 쓸데없는 열 제거
for i in range(len(civil_law)):
  civil_law.요약.values[i]= spacing(civil_law.요약.values[i])
civil_law

,원문,요약
0,제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하...,제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하...
1,제579조 채권 매매와 매도인의 담보책임채권의 매도인이 채무자의 자력을 담보한 때에...,변제기에 도달하지 아니한 채권의 매도인이 채무자의 자력을 담보한 때에는 매매 계약 ...
2,제688조 수임인의 비용 상환청구권 등 수임인이 위임사무의 처리에 관하여 필요비를 ...,제688조 수임인 이 위임사무의 처리에 필요한 채무를 부담한 때에는 위임인 에 대하...
3,제727조종신 정기금 계약의 해제 정기금 채무자가 정기금 채무의원본을 받은 경우에 ...,정기금 계약의 해제 정기금 채무자가 정기금 채무의원본을 받은 경우에 그 정기금 채무...
4,제758조 공작물 등의 점유자 소유자의 책임 공작물의 설치 또는 보존의 하자로 인하...,제758조 공작물 등의 점유자 소유자의 책임 공작물의 설치 또는 보존의 하자로 인하...
5,제410조 1인의 채권자에 생긴 사항의 효력 전조의 규정에 의하여 모든 채권자에게 ...,제410조 1인의 채권자에 생긴 사항의 효력 전조의 규정에 의하여 모든 채권자에게 ...
6,제444조 부탁 없는 보증인의 구상권 주채무자의 부탁 없이 보증인이 된 자가 변제기...,제444조 부탁 없는 보증인의 구상권 주채무자의 의사에 반하여 보증인이 된 자가 변...
7,제594조 환매의 실행매도인은 기간 내에 대금과 매매비용을 매수인에게 제공하지 아니...,환매의 실행 매도인은 기간 내에 대금과 매매 비용을 매수인에게 제공하지 아니하면 환...
8,제484조 대위변제와 채권증서 담보물 채권 전부의 대위변제를 받은 채권자는 그 채권...,제484조 대위변제와 채권 증서 담보물 채권 전부의 대위 변제를 받은 채권자는 그 ...
9,제13조 피한 정후견인의 행위와 동의가 정법원은 피한 정후견인이 한정후견인의 동의를...,가정법원은 본인 배우자 4촌 이내의 친족한 정후견인 한정후견 감독인 검사 또는 지방...


#### 순화 용어 다운로드

- 새로운 딕셔너리 dict_df 생성  
purified_dict와 새로운 순화 사전 dictionary를 합쳐서 dict_df로 통합 저장  
dict_df 기준으로 pur_df 합치기

In [26]:
dict_df=pd.DataFrame(columns=["원래  표현","변경된  표현"])
for i in range(30, 100):
  try:  
    dictionary= pd.read_excel('순화 사전.xlsx', sheet_name = "Table "+str(i))
    dictionary= dictionary[{"원래  표현", "변경된  표현"}]
    dictionary=dictionary.dropna(axis=0)
    dict_df=pd.concat([dict_df,dictionary])
  except:
    continue
dict_df.columns=["law_word","purified_word"] #pur_df와 컬럼명 통일
dict_df.to_csv("dict_df.csv")
dict_df=pd.read_csv('dict_df.csv')

In [27]:
dict_df=pd.read_csv('dict_df.csv')
dict_df.drop("Unnamed: 0",axis=1, inplace= True)
pur_df= pd.read_csv("purified_dict.csv", encoding="cp949")
dict_df=pd.concat([dict_df,pur_df], ignore_index=True) #  ignore_index:concat 시 index 리셋
dict_df.columns=["law_word","pur_word"]
dict_df

,law_word,pur_word
0,가선,전선설치
1,모선가선,모선(母線)설치
2,가점,가산점
3,처분기간을 가중 하거나 감경할 수 있다,처분기간을 늘리 거나 줄일 수 있 다
4,간명,간단명료
...,...,...
3078,휴대하다,지니다
3079,휴지하다,일시적으로 중단하다
3080,흉장,가슴표장
3081,흠결,흠


In [28]:
dict_df.drop(dict_df.index[2833], inplace=True)
dict_df.reset_index(inplace=True)
dict_df.drop("index",axis=1, inplace= True)
dict_df

,law_word,pur_word
0,가선,전선설치
1,모선가선,모선(母線)설치
2,가점,가산점
3,처분기간을 가중 하거나 감경할 수 있다,처분기간을 늘리 거나 줄일 수 있 다
4,간명,간단명료
...,...,...
3077,휴대하다,지니다
3078,휴지하다,일시적으로 중단하다
3079,흉장,가슴표장
3080,흠결,흠


한글과 띄어쓰기를 제외한 특수문자 제거

In [29]:
import re  # 정규식 표현 모듈 re

# 한글과 띄어쓰기 외 다른 요소 제거 함수 spec_words 생성
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

In [30]:
civil_law.요약[0]


'제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있고, 전항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있으며, 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고 지하지 아 니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.'

In [31]:
for n in range(len(civil_law)):
  morph= okt.morphs(civil_law.요약[n]) # okt 형태소 분석기로 형태소 단위로 추출

  # 문장에 법률용어 단어가 존재한다면 purified word로 대체해준다. 
  # n번째 word가 법률 용어라면 법률 용어의 위치와  
  for i in range(len(morph)): # posing의 각 요소 단위 별로 교체 여부 판단하기 위해 반목문의 range는 posing의 요소의 개수

    for k in range(len(dict_df)): # 순화 용어 사전의 모든 단어와 일치 여부 판단

      if morph[i] == dict_df["law_word"].values[k]: # 형태소 단위의 단어가 법률 용어라면, 
        print('\033[91m'+morph[i]+'\033[0m')
        morph[i] = dict_df["pur_word"].values[k] #  순화 용어로 바꿔준다.
        print('\033[94m'+dict_df["pur_word"].values[k]+'\033[0m')
  new_morph= spec_word(str(morph)) # 특수문자 제거
  new_morph= new_morph.replace(" ","") # 불필요한 띄어쓰기 제거
  new_morph= spacing(new_morph) # 자동 띄어쓰기

  print("원문"+str(n)+":",civil_law.원문[n])
  print("요약"+str(n)+":",civil_law.요약[n])
  print("순화:",new_morph)

전
밭
전
밭
대하
빌려줌
전
밭
흠결
흠
이를
목적어가 따로 있으면 ‘이를’을 삭제함
흠결
흠
대하
빌려줌
원문0: 제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있다. 전 항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있다. 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고지하지 아니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.
요약0: 제578조 경매와 매도인의 담보책임 경매의 경우에는 경락인은 전 8조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있고, 전항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 대하여 그 대금 전부나 일부의 반환을 청구할 수 있으며, 전 2항의 경우에 채무자가 물건 또는 권리의 흠결을 알고 고 지하지 아 니하거나 채권자가 이를 알고 경매를 청구한 때에는 경락인은 그 흠결을 안 채무자나 채권자에 대하여 손해배상을 청구할 수 있다.
순화: 제조경매와 매도인의 담보 책임 경매의 경우에는 경락인은 밭조의 규정에 의하여 채무자에게 계약의 해제 또는 대금 감액의 청구를 할 수 있고 밭항의 경우에 채무자가 자력이 없는 때에는 경락인은 대금의 배당을 받은 채권자에 빌려줌여 그 대금 전부나 일부의 반환을 청구할 수 있으며 밭항의 경우에 채무자가 물건 또는 권리의 흠을 알고 고지하지 아니하거나 채권자가 목적어가 따로 있으면 이를 을 삭제함 알고 경매를 청구한 때에는 경락인은 그 흠을 안 채무자나 채권자에 빌려줌 여 손해배상 을 청구할 수 있다
원문1: 제579조 채권 매매와 매도인의 담보책임채권의 매도인이 채무자의 자력을 담보한 때에는 매매계약 당시의 자력을 담보한 것으로 추정한다. 변제기에 도달하지 아니한 채권의 매도인이 채무자